In [2]:
import requests
from bs4 import BeautifulSoup

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [3]:
r = requests.get('https://myjgeosc.com/archive/')

In [6]:
soup = BeautifulSoup(r.content)
a = soup.find_all('a')
pages = [a_.get('href') for a_ in a if '/mjg-' in a_.get('href') or '/mjg.' in a_.get('href')]
pages = [a_ for a_ in pages if '/wp-content/uploads/' not in a_ and 'doi.org/' not in a_]
pages = sorted(list(set(pages)))
len(pages)

101

In [7]:
pages

['https://myjgeosc.com/mjg-01-2020-38-42/',
 'https://myjgeosc.com/mjg-01-2021-01-05/',
 'https://myjgeosc.com/mjg-01-2021-06-11/',
 'https://myjgeosc.com/mjg-01-2021-12-21/',
 'https://myjgeosc.com/mjg-01-2021-22-30/',
 'https://myjgeosc.com/mjg-01-2021-31-34/',
 'https://myjgeosc.com/mjg-01-2021-35-40/',
 'https://myjgeosc.com/mjg-01-2022-01-07/',
 'https://myjgeosc.com/mjg-01-2022-08-11/',
 'https://myjgeosc.com/mjg-01-2022-12-18/',
 'https://myjgeosc.com/mjg-01-2022-19-28/',
 'https://myjgeosc.com/mjg-01-2022-29-35/',
 'https://myjgeosc.com/mjg-01-2022-36-44/',
 'https://myjgeosc.com/mjg-01-2022-45-52/',
 'https://myjgeosc.com/mjg-01-2023-01-07/',
 'https://myjgeosc.com/mjg-01-2023-08-17/',
 'https://myjgeosc.com/mjg-01-2023-18-21/',
 'https://myjgeosc.com/mjg-01-2023-22-30/',
 'https://myjgeosc.com/mjg-01-2023-31-38/',
 'https://myjgeosc.com/mjg-01-2023-39-43/',
 'https://myjgeosc.com/mjg-02-2020-54-58/',
 'https://myjgeosc.com/mjg-02-2020-59-64/',
 'https://myjgeosc.com/mjg-02-20

In [8]:
from tqdm import tqdm

urls = []
for p in tqdm(pages):
    r = requests.get(p)
    soup = BeautifulSoup(r.content)
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if '/download/' in a_.get('href')]
    urls.extend(a)
    
urls

100%|█████████████████████████████████████████| 101/101 [01:15<00:00,  1.34it/s]


['https://myjgeosc.com/download/1363',
 'https://myjgeosc.com/download/1436',
 'https://myjgeosc.com/download/1438',
 'https://myjgeosc.com/download/1445',
 'https://myjgeosc.com/download/1469',
 'https://myjgeosc.com/download/1483',
 'https://myjgeosc.com/download/1489',
 'https://myjgeosc.com/download/1550',
 'https://myjgeosc.com/download/1561',
 'https://myjgeosc.com/download/1566',
 'https://myjgeosc.com/download/1577',
 'https://myjgeosc.com/download/1610',
 'https://myjgeosc.com/download/1678',
 'https://myjgeosc.com/download/1727',
 'https://myjgeosc.com/download/1741',
 'https://myjgeosc.com/download/1751',
 'https://myjgeosc.com/download/1768',
 'https://myjgeosc.com/download/1776',
 'https://myjgeosc.com/download/1785',
 'https://myjgeosc.com/download/1792',
 'https://myjgeosc.com/download/1228',
 'https://myjgeosc.com/download/1256',
 'https://myjgeosc.com/download/1292',
 'https://myjgeosc.com/download/1294',
 'https://myjgeosc.com/download/1309',
 'https://myjgeosc.com/do

In [11]:
urls = sorted(list(set(urls)))
len(urls)

100

In [12]:
!mkdir myjgeosc.com

mkdir: cannot create directory ‘myjgeosc.com’: File exists


In [13]:
import os

pdfs = urls
for i in tqdm(range(len(pdfs))):
    filename = os.path.join('myjgeosc.com', f'{i}.pdf')
    if os.path.exists(filename):
        continue
        
    u = pdfs[i]
        
    r = requests.get(u)
    with open(filename, 'wb') as fopen:
        fopen.write(r.content)

100%|█████████████████████████████████████████| 100/100 [01:57<00:00,  1.17s/it]


In [14]:
from glob import glob
from tika import parser
import json

TIKA_HOST = 'http://localhost:9998'

In [15]:
files = glob('myjgeosc.com/*.pdf')

with open('myjgeosc.com.jsonl', 'w') as fopen:
    for f in tqdm(files):
        try:
            raw_xml = parser.from_file(f, TIKA_HOST, xmlContent=True)
            body = BeautifulSoup(raw_xml['content']).find('body')
            t = '\n'.join(body.findAll(text=True))
            fopen.write(f'{json.dumps(t)}\n')
        except:
            pass

100%|█████████████████████████████████████████| 100/100 [00:06<00:00, 16.02it/s]
